### <center>**Working with Vector stores and Retrievers</centers>**
#### 06-02-2025

## These are designed to support retreival of data from vectors 
#### They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.
### We will cover
- Documents

- Vector stores

- Retrievers

### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content.
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [30]:
from langchain_core.documents import Document

document = [
    Document(
        page_content="Dogs are th great companions, known for their loyalty and friendliness.",
        metadata = {"source":"mammel-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source":"mammel-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relativly simple care.",
        metadata = {"source":"mammel-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birrds capable of mimicking Human Speech.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Rabbits are scocial animals that need plenty of space to hop around",
        metadata={"source": "mammal-pets-doc"},
    )
]

In [31]:
document

[Document(metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammel-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relativly simple care.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Parrots are intelligent birrds capable of mimicking Human Speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are scocial animals that need plenty of space to hop around')]

In [32]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model="llama-3.3-70b-specdec")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x35fcdbe20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x35fd75210>, model_name='llama-3.3-70b-specdec', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [33]:
from langchain.embeddings import HuggingFaceEmbeddings
# Here we are using "all-MiniLM-L6-v2" because sentence embedding model that can be used for tasks like semantic search, clustering, and information retrieval 
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [34]:
## VectorStores
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(document, embedding=embeddings)
vectorstore

In [35]:
## with the help of vectors
vectorstore.similarity_search("cat")

[Document(id='41af352c-7693-44b5-a192-50baf506f7a3', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='130f89f3-b8f7-4571-9e74-5b58917f2bd4', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='8224cf44-8615-44fe-90a8-21898d00388a', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.'),
 Document(id='fa0313bc-f71c-42db-94f2-8779f47afd6f', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.')]

In [36]:
## Async Query
await vectorstore.asimilarity_search("cat")

[Document(id='41af352c-7693-44b5-a192-50baf506f7a3', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='130f89f3-b8f7-4571-9e74-5b58917f2bd4', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='8224cf44-8615-44fe-90a8-21898d00388a', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.'),
 Document(id='fa0313bc-f71c-42db-94f2-8779f47afd6f', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.')]

In [37]:
## Using Similarity search with score
vectorstore.similarity_search_with_score("cat")

[(Document(id='41af352c-7693-44b5-a192-50baf506f7a3', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351049661636353),
 (Document(id='130f89f3-b8f7-4571-9e74-5b58917f2bd4', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351049661636353),
 (Document(id='8224cf44-8615-44fe-90a8-21898d00388a', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.'),
  1.5319287776947021),
 (Document(id='fa0313bc-f71c-42db-94f2-8779f47afd6f', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.'),
  1.5319287776947021)]

## Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.
LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains

In [38]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriver= RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriver.batch(["cat","dog"])

[[Document(id='41af352c-7693-44b5-a192-50baf506f7a3', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='fa0313bc-f71c-42db-94f2-8779f47afd6f', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [39]:
retriver = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriver.batch(["cat","dog"])

[[Document(id='41af352c-7693-44b5-a192-50baf506f7a3', metadata={'source': 'mammel-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='fa0313bc-f71c-42db-94f2-8779f47afd6f', metadata={'source': 'mammel-pets-doc'}, page_content='Dogs are th great companions, known for their loyalty and friendliness.')]]

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this Question using the provided context only.
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context":retriver, "question": RunnablePassthrough()}|prompt|llm

response= rag_chain.invoke("tell me about parrot and extra content")
print(response.content)

Based on the provided context, here's the information about parrots and some extra content:

Parrots are intelligent birds capable of mimicking human speech. They are known for their ability to learn and replicate words, phrases, and even entire conversations. This unique ability makes them popular pets, as they can engage in interactive conversations with their owners.

Extra content: Unfortunately, the provided context does not offer more information about parrots beyond their intelligence and ability to mimic human speech. However, it's worth noting that parrots are often colorful, social birds that thrive in flocks and require attention, care, and proper nutrition to live happy and healthy lives.
